## There are two methods that can be used to read data, which are mentioned in the following code for each method
> * 1. Creating a data frame and saving the path and label of each image in the data frame
> *   1.1 load images and labels in ram 
> *   1.2 Use flow_from_dataframe for read data
> * 2. Use flow_from_Directory for read data:
>   This method both reads the images and assigns its appropriate label to each image
   

In [70]:
'''
Creating a data frame and saving the path and label of each image in the data frame
'''
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm 
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE=32
IMG_HEIGHT = 224
IMG_WIDTH = 224
CHANNELS = 3
path = '/kaggle/input/brain-tumor-classification-mri/'
class_label= ["no_tumor", "pituitary_tumor", "meningioma_tumor", "glioma_tumor"]

def generate_datafram(path):
    train_labels = []
    train_images = []

    test_labels = []
    test_images = []

    for sub_dir,dir,files in os.walk(path):
        for file in files:
            img_path = os.path.join(sub_dir,file)
            if img_path.split('/')[-3] == 'Training':
                train_images.append(img_path)
                train_labels.append(img_path.split('/')[-2])
            elif img_path.split('/')[-3] == 'Testing':
                test_images.append(img_path)
                test_labels.append(img_path.split('/')[-2])
    train_df = pd.DataFrame(list(zip(train_images,train_labels)),columns = ['images','labels'])
    test_df = pd.DataFrame(list(zip(test_images,test_labels)),columns = ['images','labels'])
    return train_df,test_df


In [71]:
'''
 1.1  load all images in ram
'''
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    return img
def load_data_toram(train_df,test_df):
    '''   save all images in one variable '''
    N = train_df.shape[0]
    x_train = np.empty((N, IMG_WIDTH, IMG_HEIGHT, CHANNELS), dtype=np.uint8)
    for i, image_id in enumerate(tqdm(train_df['images'])): 
        x_train[i, :, :, :] = preprocess_image(
            f'{image_id}'
        )
    N_t = test_df.shape[0]
    x_test = np.empty((N_t, IMG_WIDTH, IMG_HEIGHT, CHANNELS), dtype=np.uint8)
    for i, image_id in enumerate(tqdm(test_df['images'])):
        x_test[i, :, :, :] = preprocess_image(
            f'{image_id}'
        )
    '''   converting label to one-hot encoding '''
    y_train = pd.get_dummies(train_df['labels']).values
    y_test = pd.get_dummies(test_df['labels']).values
    
    return x_train,x_test,y_train,y_test

In [72]:
'''
 1.2 Use flow_from_dataframe for read data
'''

def Read_flow_from_dataframe(train_df,test_df):
    train_datagen= ImageDataGenerator()
    train_set= train_datagen.flow_from_dataframe(train_df,
                                               x_col='images',
                                               y_col='labels',
                                               target_size=(224, 224),
                                               classes=class_label,
                                               shuffle=True,
                                               batch_size=BATCH_SIZE,
                                               class_mode='categorical')

    # test_datagen= ImageDataGenerator(rescale = 1./255)
    test_datagen= ImageDataGenerator()

    test_set= test_datagen.flow_from_dataframe(test_df,
                                               x_col='images',
                                               y_col='labels',
                                               target_size=(224, 224),
                                               classes=class_label,
                                               shuffle=True,
                                               batch_size=BATCH_SIZE,
                                               class_mode='categorical')
    return train_set,test_set

In [73]:
'''
 2 Use Read_flow_from_directory for read data
'''
def Read_flow_from_directory(path):

    train_generator = train_datagen.flow_from_directory(os.path.join(path,'Training'),
                                                        batch_size = BATCH_SIZE, 
                                                        class_mode = 'categorical',
                                                        classes=class_label,
                                                        target_size = (224, 224))

    test_generator = test_datagen.flow_from_directory( os.path.join(path,'Testing'),
                                                       shuffle=False,
                                                       batch_size = BATCH_SIZE,
                                                       class_mode = 'categorical',
                                                       classes=class_label,
                                                       target_size = (224, 224))
    return train_generator,test_generator

In [74]:
'''
  choose one method for read your data
'''

def load_data(method):
    
    if method=='load_to_ram':
        train_df,test_df=generate_datafram(path)
        x_train,x_test,y_train,y_test = load_data_toram(train_df,test_df)
        return x_train,x_test,y_train,y_test
    if method=='Read_from_dataframe':
        train_df,test_df=generate_datafram(path)
        train_set,test_set = Read_flow_from_dataframe(train_df,test_df)
        return train_set,test_set
    if method=='Read_use_directory':
        train_set,test_set = Read_flow_from_directory(path)
        return train_set,test_set


In [83]:
def main():
    method = 'Read_from_dataframe'
    if method == 'load_to_ram':
        x_train,x_test,y_train,y_test  = load_data('load_to_ram')
    if method == 'Read_from_dataframe':
        train_set,test_set  = load_data('Read_from_dataframe')
    if method == 'Read_use_directory':
        train_set,test_set  = load_data('Read_use_directory')
main()

Found 2870 validated image filenames belonging to 4 classes.
Found 394 validated image filenames belonging to 4 classes.
